<a href="https://colab.research.google.com/github/henriqueyjw/PROSECUTORIAL_INDEPENDENCE/blob/main/identificacao_politicos_Pesquisa_FabioKercher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pesquisa Partes Interesse STF

Objetivo: Identificar, dada uma planilha de pessoas que sofreram processo pelo PGR, quais delas são políticos.

## Configurações

In [ ]:
pip install rapidfuzz

In [ ]:
import pandas as pd
import glob
import os
import unicodedata
from rapidfuzz import process, fuzz
import re

In [ ]:
def remover_acentos(texto):
    if not isinstance(texto, str):
        return texto
    nfkd = unicodedata.normalize('NFKD', texto)
    return ''.join([c for c in nfkd if not unicodedata.combining(c)])

def remover_preposicoes(nome):
    """
    Remove as preposições 'de', 'das', 'dos', 'do', 'da' de um nome.
    """
    if not isinstance(nome, str):
        return nome
    # Regex: ^|\s para pegar no início ou com espaço antes, (?i) para ignorar maiúsculas/minúsculas
    nome_sem_preposicoes = re.sub(r"(?i)\b(de|das|dos|do|da)\b\s*", "", nome.lower()).strip()
    return nome_sem_preposicoes.upper()

def limpar_espacos(nome):
    """
    Remove espaços extras entre nomes e no início/fim da string.
    """
    if not isinstance(nome, str):
        return nome
    # Divide por espaços, remove strings vazias e junta as palavras com um único espaço
    return ' '.join(nome.split())

def normalizar_nome(nome):
    if not isinstance(nome, str):
        return ''
    nome = remover_acentos(nome)
    nome = remover_preposicoes(nome)
    nome = limpar_espacos(nome)
    nome = nome.upper()
    nome = nome.strip()
    return nome

def deduplica_nomes_fuzzy(df, coluna, threshold=85):
    nomes = df[coluna].dropna().unique()
    usados = set()
    resultado = []
    for nome in nomes:
        if nome in usados:
            continue
        similares = process.extract(
            nome,
            nomes,
            scorer=fuzz.ratio,
            limit=None
        )
        # Seleciona nomes acima do limiar de similaridade
        grupo = [n for n, p, _ in similares if p >= threshold and n not in usados]
        # Mantém o nome mais longo do grupo
        nome_representante = max(grupo, key=len)
        resultado.append(nome_representante)
        usados.update(grupo)
    # Cria DataFrame apenas com nomes deduplicados
    return pd.DataFrame({coluna: resultado})

def deduplica_nomes_fuzzy_mapping(df, coluna, threshold=85):
    nomes = df[coluna].dropna().unique()
    usados = set()
    grupos = []
    for nome in nomes:
        if nome in usados:
            continue
        similares = process.extract(
            nome,
            nomes,
            scorer=fuzz.ratio,
            limit=None
        )
        # Seleciona nomes acima do limiar de similaridade
        grupo = [n for n, p, _ in similares if p >= threshold and n not in usados]
        # Mantém o nome mais longo do grupo
        nome_representante = max(grupo, key=len)
        grupos.append((grupo, nome_representante))
        usados.update(grupo)
    # Cria mapeamento para substituir
    mapping = {}
    for grupo, representante in grupos:
        for nome in grupo:
            mapping[nome] = representante
    return mapping


## Importação de Dados

Dados das Partes processadas

In [ ]:
df_interesse = pd.read_excel('partes_interesse_stf.xlsx')

In [ ]:
print(f'Ano mais antigo dos dados: {df_interesse["ano"].min()}')
print(f'Ano mais recente dos dados: {df_interesse["ano"].max()}')
print(f'Qtd de Linhas: {df_interesse.shape[0]}')

Ano mais antigo dos dados: 1995
Ano mais recente dos dados: 2023
Qtd de Linhas: 7103


Dados da Justiça Eleitoral. Todos os candidatos das eleições de 1994 até 2022

In [ ]:
# Caminho para a pasta
caminho_pasta = '/content/candidatos/'

# Lista todos os arquivos .csv na pasta
arquivos = glob.glob(os.path.join(caminho_pasta, '*.csv'))

# Lê todos os arquivos em dataframes e concatena
dfs = [pd.read_csv(arquivo, encoding='latin1', sep=';') for arquivo in arquivos]

#df final
df_eleitoral = pd.concat(dfs, ignore_index=True)

/tmp/ipython-input-120819715.py:8: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(arquivo, encoding='latin1', sep=';') for arquivo in arquivos]
/tmp/ipython-input-120819715.py:8: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(arquivo, encoding='latin1', sep=';') for arquivo in arquivos]
/tmp/ipython-input-120819715.py:8: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(arquivo, encoding='latin1', sep=';') for arquivo in arquivos]
/tmp/ipython-input-120819715.py:8: DtypeWarning: Columns (20,40) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(arquivo, encoding='latin1', sep=';') for arquivo in arquivos]


In [ ]:
print(f'Tamanho df_eleitoral antes: {df_eleitoral.shape[0]}')
df_eleitoral.drop_duplicates(subset=['NR_CPF_CANDIDATO', 'ANO_ELEICAO'], inplace=True)
print(f'Tamanho df_eleitoral depois: {df_eleitoral.shape[0]}')

Tamanho df_eleitoral antes: 2902126
Tamanho df_eleitoral depois: 2868483


## Limpeza do DF_INTERESSE

In [ ]:
df_interesse.rename(columns={'parte_interesse': 'parte_interesse_original'}, inplace=True)

df_interesse['parte_interesse'] = (
    df_interesse['parte_interesse_original']
    .str.upper()  # deixar tudo maiúsculo
    .str.split(' OU ')  # split pelo OU
    .str[0]  # pega a primeira parte
    .str.strip()  # tira espaços extras
)

# remove acentos
df_interesse['parte_interesse'] = df_interesse['parte_interesse'].apply(remover_acentos)

print(f'Qtd de Linhas (antes da remoção de empresa): {df_interesse.shape[0]}')

# Pré-processamento para maiúsculo
df_interesse['parte_interesse'] = df_interesse['parte_interesse'].str.upper()

padrao_empresa_simplificado = (
    r'REPRESENTANTE LEGAL'
    r'|S\.A'
    r'|S/A'
    r'|SINDICATO'
    r'|PARTIDO'
    r'|ASSOCIACAO'
    r'|ASSOCIAÇAO'
    r'|LTDA'
    r'|SOCIEDADE'
    r'|MINISTERIO PUBLICO'
    r'|CONGREGACAO'
    r'|CONGREGAÇAO'
    r'|FUNDACAO'
    r'|FUNDAÇAO'
    r'|FUNDACAO PUBLICA'
    r'|FUNDACAO PRIV'
    r'|SOCIEDADE CIVIL'
)

padrao_sigla = r'^([A-Z]{1,4}\.?[\s]?){2,}$'

empresa_mask = df_interesse['parte_interesse'].str.contains(padrao_empresa_simplificado, regex=True, na=False)


df_interesse = df_interesse[~(empresa_mask)]


print(f'Qtd de Linhas (depois da remoção de empresa): {df_interesse.shape[0]}')

#remove duplicados
df_interesse = df_interesse.drop_duplicates(subset='parte_interesse')

print(f'Qtd de Linhas (depois da remoção de duplicado): {df_interesse.shape[0]}')

Qtd de Linhas (antes da remoção de empresa): 7103
Qtd de Linhas (depois da remoção de empresa): 6989
Qtd de Linhas (depois da remoção de duplicado): 4431


In [ ]:
df_interesse['parte_interesse'] = df_interesse['parte_interesse'].apply(normalizar_nome)
df_interesse.drop_duplicates(subset='parte_interesse', inplace=True)
print(f'Qtd de Linhas (depois da normalização e remoção de duplicados): {df_interesse.shape[0]}')

Qtd de Linhas (depois da normalização e remoção de duplicados): 4398


In [ ]:
df_nomes = deduplica_nomes_fuzzy(df_interesse, "parte_interesse", threshold=75)
print(f'Tamanho df_nomes (remoção com fuzzy): {df_nomes.shape[0]}')

Tamanho df_nomes (remoção com fuzzy): 3508


In [ ]:
df_interesse = df_interesse[df_interesse["parte_interesse"].isin(df_nomes["parte_interesse"])].reset_index(drop=True)
print(f'Tamanho df_interesse: {df_interesse.shape[0]}')

Tamanho df_interesse: 3508


## Merge dos Dataframes com Fuzzy

In [ ]:
# Padroniza
df_nomes['nome_normalizado'] = df_nomes.iloc[:,0].apply(normalizar_nome)
df_eleitoral['nome_normalizado'] = df_eleitoral['NM_CANDIDATO'].apply(normalizar_nome)
lista_eleitoral = df_eleitoral['nome_normalizado'].dropna().unique()

resultados = []
threshold = 75  # Ajuste este valor se necessário

def fuzzy_lote(df_nomes, lista_eleitoral, threshold=75, batch_size=500):
    from rapidfuzz import process, fuzz

    nomes = df_nomes['nome_normalizado'].tolist()
    resultados = []

    for i in range(0, len(nomes), batch_size):
        nomes_batch = nomes[i:i+batch_size]
        matches = process.cdist(
            nomes_batch,
            lista_eleitoral,
            scorer=fuzz.token_sort_ratio,
            workers=-1
        )
        melhor_indices = matches.argmax(axis=1)
        melhor_scores = matches.max(axis=1)
        # Adiciona resultados do lote
        for nome, idx, score in zip(nomes_batch, melhor_indices, melhor_scores):
            if score >= threshold:
                resultados.append({
                    'nome_normalizado': nome,
                    'match_idx': idx,
                    'match_score': score,
                    'nome_match': lista_eleitoral[idx]
                })
    return pd.DataFrame(resultados)

# Batched fuzzy matching: consome MUITO MENOS RAM, ideal para grandes volumes
df_nomes_matches = fuzzy_lote(df_nomes, lista_eleitoral, threshold=threshold, batch_size=500)

# Merge com eleitoral
df_historico_politico = pd.merge(
    df_nomes_matches,
    df_eleitoral,
    left_on='nome_match',
    right_on='nome_normalizado',
    how='left'
)

In [ ]:
for nome, dados in df_historico_politico.groupby('nome_normalizado_x'):
    print(f"\nHistórico de: {nome}")
    print(dados[['ANO_ELEICAO', 'DS_CARGO', 'SG_PARTIDO', 'NM_URNA_CANDIDATO', 'NM_CANDIDATO', 'SG_UE']])

In [ ]:
df_historico_politico.to_csv('historico_politico.csv', index=False)
df_historico_politico.to_excel('df_historico_politicos.xlsx')

## Marcando como "político"

Macando como "político" e colocando histórico da última eleição no df principal

In [ ]:
df_interesse['nome_normalizado'] = df_interesse['parte_interesse'].apply(normalizar_nome)
df_historico_politico['nome_normalizado'] = df_historico_politico['NM_CANDIDATO'].apply(normalizar_nome)

# Como há múltiplos registros do mesmo político, pegamos o registro da eleição mais recente para cada nome
df_historico_politico_ult = (
    df_historico_politico.sort_values('ANO_ELEICAO', ascending=False)
    .drop_duplicates('nome_normalizado')
    .reset_index(drop=True)
)

# Prepara lista de nomes políticos para fuzzy search
nomes_politicos = df_historico_politico_ult['nome_normalizado'].tolist()

# Função fuzzy de busca do histórico do político
def busca_historico_fuzzy(nome):
    nome_norm = normalizar_nome(nome)
    melhor = process.extractOne(nome_norm, nomes_politicos, scorer=fuzz.token_sort_ratio)
    # threshold pode ser ajustado (ex: 85 ou 90)
    if melhor and melhor[1] >= 85:
        nome_politico_encontrado = melhor[0]
        poli_row = df_historico_politico_ult[df_historico_politico_ult['nome_normalizado'] == nome_politico_encontrado].iloc[0]
        return pd.Series({
            'politico': 'Sim',
            'ANO_ELEICAO': poli_row['ANO_ELEICAO'],
            'DS_CARGO': poli_row['DS_CARGO'],
            'SG_PARTIDO': poli_row['SG_PARTIDO'],
            'NM_URNA_CANDIDATO': poli_row['NM_URNA_CANDIDATO'],
            'NM_CANDIDATO': poli_row['NM_CANDIDATO'],
            'nome_politico_encontrado': nome_politico_encontrado,
            'similaridade': melhor[1]
        })
    return pd.Series({
        'politico': '',
        'ANO_ELEICAO': '',
        'DS_CARGO': '',
        'SG_PARTIDO': '',
        'NM_URNA_CANDIDATO': '',
        'NM_CANDIDATO': '',
        'nome_politico_encontrado': '',
        'similaridade': ''
    })

# Aplica fuzzy search na coluna interesse
df_interesse = df_interesse.join(df_interesse['parte_interesse'].apply(busca_historico_fuzzy))

In [ ]:
df_interesse.to_csv('df_interesse_sem_empresa_politicos.csv')
df_interesse.to_excel('df_interesse_sem_empresa_politicos.xlsx')